In [1]:
import os
import time

from web3 import HTTPProvider, Web3

from eth_defi.abi import get_contract
from eth_defi.chain import install_chain_middleware
from eth_defi.event_reader.filter import Filter
from eth_defi.event_reader.reader import read_events, LogResult
from eth_defi.event_reader.reorganisation_monitor import JSONRPCReorganisationMonitor

In [2]:
from eth_defi.abi import get_contract

In [3]:
json_rpc_url = os.environ.get("JSON_RPC_POLYGON", "https://polygon-rpc.com")
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()
install_chain_middleware(web3)

# Get contracts
Pair = get_contract(web3, "sushi/UniswapV2Pair.json")
filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
# Get the headers of last 5 blocks before starting
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
from eth_defi.event_reader.conversion import decode_data, convert_uint256_hex_string_to_address, convert_uint256_bytes_to_address, convert_int256_bytes_to_int

In [7]:
events = []

evt: LogResult
for evt in read_events(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
):
    events.append(evt)
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]

    arguments = decode_data(evt["data"])
    topics = evt["topics"]
    token0 = convert_uint256_hex_string_to_address(topics[1])
    token1 = convert_uint256_hex_string_to_address(topics[2])
    pair_address = convert_uint256_bytes_to_address(arguments[0])
    pair_id = convert_int256_bytes_to_int(arguments[1])
    
    # The reader may cause duplicate events as the chain tip reorganises
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']} pair:{pair_address}")
        processed_events.add(key)
else:
    print(".")

Swap at block:61,383,667 tx:0x7d8330927dc69a447078feaa25a2ad55a04e4464eb4786dd4b8cad7be20b972a pair:0x0000000000000000000000000000000000000000
Swap at block:61,383,667 tx:0xfab70289f933b4d988e55e3a503908d81c37019942b702d0ff7a8ee3bdbc6af4 pair:0x0000000000000000000000000000000000000000
Swap at block:61,383,668 tx:0xf17a345fd5e62c89bd3f60aa47c65c9fea92eced6c26b30d0760fe5a93be0ecc pair:0x0000000000000000000000000000000000000000
Swap at block:61,383,673 tx:0x555c3f41edc0d952c73dfa1612d302a173b9e07fd57dd5e05353f796de52c245 pair:0x0000000000000000000000002eAb5efAcD603714
Swap at block:61,383,674 tx:0xda487c1b7d30be6a691f8b01ecbd04f7a2f8baad51b401062a5df5310fcd88f6 pair:0x00000000000000000000000089f5358C4d088700
Swap at block:61,383,675 tx:0x45733703a2834a6ec232415560ce2a4aa130633d3193e4b92e2b202ceb1eb8b4 pair:0x000000000000000000000000000000000004ff1b
.


In [8]:
evt = events[2]

arguments = decode_data(evt["data"])
topics = evt["topics"]

event = {}
event['chain'] = 'polygon'
event['address'] = evt["address"]
event['tx_hash'] = evt["transactionHash"]
event['blk_num'] = evt["blockNumber"]
event['timestamp'] = evt["timestamp"]
event['event_type'] = evt["event"]
event['event_details'] = {}
event['event_details']['token0'] = convert_uint256_hex_string_to_address(topics[1])
event['event_details']['token1'] = convert_uint256_hex_string_to_address(topics[2])
event['event_details']['amount0In'] = convert_int256_bytes_to_int(arguments[0]) 
event['event_details']['amount1Out'] = convert_int256_bytes_to_int(arguments[3]) 

event

{'chain': 'polygon',
 'address': '0x032d86f50ee9c0c361f268bd7c08013465e29968',
 'tx_hash': '0xf17a345fd5e62c89bd3f60aa47c65c9fea92eced6c26b30d0760fe5a93be0ecc',
 'blk_num': 61383668,
 'timestamp': 1725384791,
 'event_type': web3._utils.datatypes.Swap,
 'event_details': {'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
  'token1': '0x8E1c68Ba7f6737E42f9f7CB39a20DF284EC58cf8',
  'amount0In': 0,
  'amount1Out': 0}}